In [4]:
from tkinter import *
import PIL
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import LinearSVC
from keras.datasets import mnist
from sklearn.metrics import accuracy_score
from sklearn import datasets, tree, metrics
from sklearn.model_selection import train_test_split
canvas_width=600
canvas_height=450
gray = (213,213,213)
def paint(event):
    x1,y1 = (event.x-10),(event.y-10)
    x2,y2=(event.x+10),(event.y+10)
    c.create_oval(x1,y1,x2,y2,fill="black",outline="black")
    draw.ellipse((x1,y1,x2,y2),fill="black",outline="black")
def xoa():
    global img,draw
    img=PIL.Image.new("RGB",(600,450),gray)
    draw = ImageDraw.Draw(img)
    c.delete("all")
    

def result():
    filename="C:/Users/HOANG NAM/Desktop/digit.jpg"
    img.save(filename)
    lbl['text']=loadimg(filename)
    

X_train, X_test, y_train, y_test = train_test_split( new_data, new_target, test_size=0.4, shuffle=False)

model = tree.DecisionTreeClassifier(criterion="entropy", random_state=33)
model.fit(X_train, y_train)

y_pre = model.predict(X_test)
print(accuracy_score(y_test,y_pre))

def loadimg(filename):
    image = cv2.imread(filename)
    im_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    im_blur = cv2.GaussianBlur(im_gray,(5,5),0)
    im,thre = cv2.threshold(im_blur,90,255,cv2.THRESH_BINARY_INV)
    contours,hierachy = cv2.findContours(thre,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    rects = [cv2.boundingRect(cnt) for cnt in contours]
    res=[]
    for i in contours:
        (x,y,w,h) = cv2.boundingRect(i)
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),3)
        roi = thre[y:y+h,x:x+w]
        roi = np.pad(roi,(20,20),'constant',constant_values=(0,0))
        roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
        roi = cv2.dilate(roi, (3, 3))
        roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1),block_norm="L2")
        nbr = model.predict(np.array([roi_hog_fd], np.float32))
        res.append(nbr)
        cv2.putText(image, str(int(nbr[0])), (x, y),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)
        #cv2.imshow("image",image)
    return res


    
master=Tk()
master.title('paint')
c=Canvas(master,width=canvas_width,height=canvas_height,bg="white")
c.pack()








c.pack(expand=YES,fill=BOTH)
c.bind('<B1-Motion>',paint)

#btsave=Button(text="Train",command=train,padx=50,fg="blue")
#btsave.pack()

btresult=Button(text="result",command=result,padx=50,fg="blue")
btresult.pack()

btclear=Button(text="Clear",command=xoa,padx=50,fg="blue")
btclear.pack()

lbl = Label(master, text="")
lbl.pack()

#message = tkinter.Lable(master,text='Press and drag mouse to draw')
#message.pack(side=BOTTOM)

master.mainloop()



TypeError: Singleton array array(<_io.TextIOWrapper name='D:/restart/new_data.txt' mode='r' encoding='cp1252'>,
      dtype=object) cannot be considered a valid collection.